In [1]:
import pandas as pd
from rdkit import Chem
import numpy as np
import time
import datetime
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import statistics

from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

import random
from deap import base, creator, tools

import warnings
warnings.filterwarnings('ignore')

In [2]:
#            Ag  Al  As  At  B  Ba  Be Bi  Br  C  Ca  Cl  Cs  F  H  He  I  K  Kr  Li  Mg  N  Na  O  P   Ra  Rb  S   Sc  Sn  Te  Xe  Zn]
atoms_num = [47, 13, 33, 85, 5, 56, 4, 83, 35, 6, 20, 17, 55, 9, 1, 2, 53, 19, 36, 3, 12, 7, 11, 8, 15, 88, 37, 16, 21, 50, 52, 54, 30] 

In [3]:
def balance_parenthesis_type(smiles, parenthesis):
    stack = [] 
    balanced_smiles = ''
    
    for char in smiles:
        if char not in parenthesis:
            balanced_smiles += char
        elif char in parenthesis[0]:
            stack.append(char)
            balanced_smiles += char 
        elif char in parenthesis[1]:
            if not stack:
                balanced_smiles += parenthesis[0]
            else:
                stack.pop()
            balanced_smiles += char 
        
    while stack:
        stack.pop()
        balanced_smiles += parenthesis[1]
    return balanced_smiles

def balance_parenthesis(smiles):
    smiles = balance_parenthesis_type(smiles,'()')
    balanced_smiles = balance_parenthesis_type(smiles,'[]')
    balanced_smiles = balanced_smiles.replace('()','').replace('[]','')
    return balanced_smiles

In [4]:
def my_mutation(individual):
    for _ in range(30):
        try:
            mol = Chem.MolFromSmiles(individual)
            mol = Chem.RWMol(mol)
            max_len = mol.GetNumAtoms()
            rnd_atom = np.random.randint(0, max_len-1)
            if mol.GetAtomWithIdx(rnd_atom).GetIsAromatic():
                mol.ReplaceAtom(rnd_atom, Chem.Atom(random.choice([6, 7, 15, 8, 16])))
            else:
                atom_index = np.random.randint(0, len(atoms_num)-1)
                mol.ReplaceAtom(rnd_atom, Chem.Atom(atoms_num[atom_index]))

            new_individual = Chem.MolToSmiles(mol)
        except:
            new_individual = '()'
        if Chem.MolFromSmiles(new_individual) is not None: 
            mol = Chem.MolFromSmiles(new_individual)
            new_individual = Chem.MolToSmiles(mol)
            if new_individual != individual:
                return new_individual,
    return None,

In [5]:
def my_crossover(parent_a,parent_b):
    check = []
    i = 0
    for _ in range(30):
        cut_point_a = random.randint(0, len(parent_a) - 1)
        cut_point_b = random.randint(0, len(parent_b) - 1)
        a1 = parent_a[0:cut_point_a]
        a2 = parent_a[cut_point_a:len(parent_a)]
        b1 = parent_b[0:cut_point_b]
        b2 = parent_b[cut_point_b:len(parent_b)]
        child1 = a1 + b2
        child2 = b1 + a2
        child1 = balance_parenthesis(child1)
        child2 = balance_parenthesis(child2)
        if Chem.MolFromSmiles(child1) is not None and Chem.MolFromSmiles(child2) is not None:
            ch1 = child1
            ch2 = child2
            mol1 = Chem.MolFromSmiles(child1)
            mol2 = Chem.MolFromSmiles(child2)
            child1 = Chem.MolToSmiles(mol1)
            child2 = Chem.MolToSmiles(mol2)
            return child1, child2
    for _ in range(30):
        cut_point_a = random.randint(0, len(parent_a) - 1)
        cut_point_b = random.randint(0, len(parent_b) - 1)
        a1 = parent_a[0:cut_point_a]
        a2 = parent_a[cut_point_a:len(parent_a)]
        b1 = parent_b[0:cut_point_b]
        b2 = parent_b[cut_point_b:len(parent_b)]
        child1 = b2 + a1
        child2 = a2 + b1
        child1 = balance_parenthesis(child1)
        child2 = balance_parenthesis(child2)
        if Chem.MolFromSmiles(child1) is not None and Chem.MolFromSmiles(child2) is not None:
            ch1 = child1
            ch2 = child2
            mol1 = Chem.MolFromSmiles(child1)
            mol2 = Chem.MolFromSmiles(child2)
            child1 = Chem.MolToSmiles(mol1)
            child2 = Chem.MolToSmiles(mol2)
            return child1, child2
    return None, None

In [6]:
def read_file(file_name):
    smiles_list = []
    with open(file_name,'r') as file:
        for smiles in file:
            smiles = smiles.replace('\n', '')
            smiles_list.append(smiles)

    return smiles_list

random_molecules = read_file('random_SMILES.txt')
#random_molecules = read_file('SMILES_high.txt')

In [7]:
from rdkit import Chem
from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer


def evaluate_individual(individual):
    mol = Chem.MolFromSmiles(individual)
    score = sascorer.calculateScore(mol)
    return score,  

In [8]:
from rdkit.Chem import QED
def qed_score(mol):
    mol = Chem.MolFromSmiles(mol)
    score = QED.default(mol)
    return score

In [9]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", str, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_molecule", random.choice, random_molecules)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_molecule, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate_individual)
toolbox.register("mutate", my_mutation)
toolbox.register("mate", my_crossover)
toolbox.register("select", tools.selTournament, tournsize=5)


In [10]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("mean", lambda x: sum(x) / len(x))
stats.register("std", lambda x: round(statistics.pstdev(x),3))
stats.register("min", lambda x: min(x))
stats.register("max", lambda x: max(x))

# GA

In [11]:
population = [creator.Individual(toolbox.attr_molecule()) for _ in range(10)]
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit
fits = [ind.fitness.values[0] for ind in population]


offspring = toolbox.select(population, len(population))
offspring = list(map(toolbox.clone, offspring))

fits

[3.774091125209239,
 3.9158621764687656,
 4.1782883149178645,
 6.243111097975944,
 2.98883590385498,
 6.265500311793238,
 3.2993667858496636,
 4.4885981682900296,
 4.478888314917865,
 2.8422173048844197]

In [12]:
ngen = 20
cxpb = 1
mutpd = 0.05


for i in range(1, ngen+1): # From DEAP eaSimple
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))
    
    for i in range(1, len(offspring), 2):
        if random.random() < cxpb:
            offspring[i - 1], offspring[i] = toolbox.mate(offspring[i - 1],
                                                          offspring[i])
            del offspring[i - 1].fitness.values, offspring[i].fitness.values

    for i in range(len(offspring)):
        if random.random() < mutpb:
            offspring[i], = toolbox.mutate(offspring[i])
            del offspring[i].fitness.values
            
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    population[:] = offspring

AttributeError: 'str' object has no attribute 'fitness'

# Step by step break down

In [13]:
population = [creator.Individual(toolbox.attr_molecule()) for _ in range(10)]
population

['CC(N=O)OCCCCNCCPC=O',
 'CC=C1C(=O)CC1C',
 'CCCC(O)C(C)=C1C=C=CC1C=O',
 'NN1C=CC=C=CC(NCC=CC=O)CO1',
 'C=[SH]CN=C(CCCC=O)CC(N=O)NC',
 'C#CCCC(=O)C(C)C',
 'CC=CC#CCCC=CCC=CC=O',
 'O=CCC1COC1',
 'CC1N=CC=CN2C(CCC=C=O)=C12',
 'CCCC(O)C(N)CCCl']

In [14]:
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit
fits = [ind.fitness.values[0] for ind in population]
fits

[4.582227383580593,
 3.819725686948564,
 5.2646687197574185,
 5.801631205815811,
 5.182245602211745,
 2.8422173048844197,
 3.7312513549229074,
 3.000307797050394,
 5.157060784534748,
 3.588170393920489]

In [15]:
offspring = toolbox.select(population, len(population))
offspring = list(map(toolbox.clone, offspring))
offspring

['O=CCC1COC1',
 'CCCC(O)C(N)CCCl',
 'C#CCCC(=O)C(C)C',
 'CC=CC#CCCC=CCC=CC=O',
 'CCCC(O)C(N)CCCl',
 'C#CCCC(=O)C(C)C',
 'CCCC(O)C(N)CCCl',
 'O=CCC1COC1',
 'O=CCC1COC1',
 'O=CCC1COC1']

#### ERROR: after mating type changes from <class 'deap.creator.Individual'> to <class 'str'>, which is not happenig if you use predefined crossover (which does not fit my problem)

In [16]:
for i in range(1, len(offspring), 2):
        if random.random() < cxpb:
            print(f'Selected for mating: {offspring[i - 1]}, {offspring[i]}')
            print(f'Type of offspring[i] is: {type(offspring[i])}')
            offspring[i - 1], offspring[i] = toolbox.mate(offspring[i - 1],
                                                          offspring[i])
            print(f'After mating: {offspring[i - 1]} , {offspring[i]}')
            print(f'Type of offspring[i] after mating is: {type(offspring[i])}')
                                              
            del offspring[i - 1].fitness.values, offspring[i].fitness.values

Selected for mating: O=CCC1COC1, CCCC(O)C(N)CCCl
Type of offspring[i] is: <class 'deap.creator.Individual'>
After mating: NC(CCCl)OO , CCCC=CCC1COC1
Type of offspring[i] after mating is: <class 'str'>


AttributeError: 'str' object has no attribute 'fitness'